# IIC2433 - Proyecto semestral


Este notebook se creó usando Google Colab, en esta plataforma es necesario correr la siguiente celda para forzar la actualización de la librería plotly. De otra forma, no se podrán ejecutar correctamente las celdas correspondientes a la Visualización en Mapas

In [ ]:
pip install -U plotly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from functools import reduce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## 1. Definición del problema y su impacto


Queremos responder a la pregunta: "Qué comunas se comportan de manera similar en su respuesta al COVID-19?". Para esto usaremos datos que entregan información respecto a la evolución y respuesta de COVID-19 en cada comuna de Chile. Estos datos provienen del [repositorio del Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19). 

Encontrar la respuesta a este problema puede ser de utilidad para las autoridades del Ministerio de Salud y el Ministerio de Ciencia, que utilizan la evolución de casos por comuna y la tasa de incidencia para decretar si una comuna debe presentar modificaciones en el plan Paso a Paso. 

Nuestra hipótesis es que se pueden formar grupos de comunas que han tenido un comportamiento similar en sus cifras de COVID-19. Además de los casos totales y la tasas de incidencia se podrían considerar otros factores, como la cobertura del testeo y la cantidad de tests por búsqueda activa de casos.

Estos nuevos grupos de comportamiento similar pueden ser utilizados como las unidades territoriales a las que se aplican las medidas sanitarias del plan Paso a Paso, permitiendo así que cada comuna tenga las medidas sanitarias más adecuadas a su comportamiento histórico.

El análisis lo vamos a realizar con todas las comunas del país, pero también queremos analizar solo las comunas de la región metropolitana.

## 2. Datos

Descargamos los datos de COVID-19, están disponibles en un gist de github, pero realmente vienen de un [repositorio del Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19). A continuación se presentan los links a las descripciones de los datos a usar: 

- [Casos totales por comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1)
- [Movilidad en chile por comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto33)
- [Fallecidos por comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto38)
- [índice de positividad test RT-PRC por comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto65)
- [Cobertura de testeo por comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto66)
- [Cantidad de Test por búsqueda activa de casos (BAC) por Comuna](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto64)

Los archivos csv que serán utilizados se pueden encontrar en las url de las siguientes celdas:


In [ ]:
url_total_cases = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv'
url_mobility = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto33/IndiceDeMovilidad-IM.csv'
url_deceased = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv'
url_PCR_positivity = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto65/PositividadPorComuna.csv'
url_test_coverage = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto66/CoberturaPorComuna.csv'
url_BAC = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto64/BACPorComuna.csv'

## 3. Preprocesamiento

Definimos los dataframes

In [ ]:
df_total_cases = pd.read_csv(url_total_cases)
df_mobility = pd.read_csv(url_mobility)
df_deceased = pd.read_csv(url_deceased)
df_PCR_positivity = pd.read_csv(url_PCR_positivity)
df_test_coverage = pd.read_csv(url_test_coverage)
df_BAC = pd.read_csv(url_BAC)

Realizamos una función para agrupar columas de acuerdo a la fecha

In [ ]:
def create_regular_expressions():
  years = ['2020', '2021']
  months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  regex_expressions = []
  for year in years:
    for month in months:
      regex_expressions.append(year + '-' + month + '-.*')
  return regex_expressions

def group_date_columns(df, column_name):
  reg_expressions = create_regular_expressions()
  # new_df = pd.DataFrame()
  #new_df['comuna'] = df['Comuna']
  new_df = df[['Region', 'Comuna', 'Codigo comuna', 'Poblacion']]
  
  for exp in reg_expressions:
    # Creamos las columnas nuevas con el promedio del mes
    new_df[column_name + '_' + exp[:7]] = df.filter(regex=exp).mean(axis=1)
  
  # Eliminamos las fechas en que no tenemos datos
  new_df.drop(columns=[f"{column_name}_2020-01", f"{column_name}_2020-02", f"{column_name}_2021-11", f"{column_name}_2021-12"], inplace=True)
  # Los demás promedios los llenamos con ceros
  new_df.fillna(0, inplace=True)
  return new_df

Agrupamos los datos de fechas de cada df


In [ ]:
df_total_cases_info = group_date_columns(df_total_cases, 'total_cases')
df_mobility_info = group_date_columns(df_mobility, 'mobility')
df_deceased_info = group_date_columns(df_deceased, 'deceased')
df_PCR_positivity_info = group_date_columns(df_PCR_positivity, 'PRC_positivity')
df_test_coverage_info = group_date_columns(df_test_coverage, 'test_coverage')
df_BAC_info = group_date_columns(df_BAC, 'BAC')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Hacemos merge de los df por comuna para obtener el df con toda la información


In [ ]:
dfs = [df_total_cases_info, df_mobility_info, df_deceased_info, df_PCR_positivity_info, df_test_coverage_info, df_BAC_info]
df_merged = reduce(lambda left,right: pd.merge(left, right, on=['Codigo comuna'], how='inner'), dfs)
df_merged

,Region_x,Comuna_x,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Region_y,Comuna_y,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,mobility_2020-12,mobility_2021-01,mobility_2021-02,mobility_2021-03,...,test_coverage_2020-06,test_coverage_2020-07,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,test_coverage_2021-08,test_coverage_2021-09,test_coverage_2021-10,Region_y,Comuna_y,Poblacion_y,BAC_2020-03,BAC_2020-04,BAC_2020-05,BAC_2020-06,BAC_2020-07,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07,BAC_2021-08,BAC_2021-09,BAC_2021-10
0,Arica y Parinacota,Arica,15101.0,247552.0,6.0,112.909091,406.888889,1155.875,3159.444444,6042.000000,7737.250,9294.222222,10332.222222,10842.125,11984.444444,14315.000,17066.000000,20570.111111,23859.000000,26726.375,28812.000000,29463.666667,29905.000,30671.444444,Arica y Parinacota,Arica,247552.0,5.394165,4.092249,4.283068,4.973705,5.049779,5.235622,5.868555,5.983744,6.312275,0.0,0.0,0.0,0.0,...,0.0,0.0,95.35,95.825,94.800,94.66,94.400,93.675,95.625,94.36,92.075,93.78,88.825,90.650,0.0,0.0,0.0,Arica y Parinacota,Arica,247552.0,0.0,0.0,0.0,0.0,0.0,60.60,66.975,64.700,73.38,78.525,81.650,83.025,84.74,85.300,87.16,86.425,87.700,0.0,0.0,0.0
1,Arica y Parinacota,Camarones,15102.0,1233.0,0.0,0.000000,0.000000,0.000,8.666667,24.333333,26.375,27.888889,28.000000,27.500,28.555556,30.375,39.666667,50.777778,57.222222,61.375,65.444444,67.444444,68.500,70.888889,Arica y Parinacota,Camarones,1233.0,19.541699,7.755541,6.572208,8.568462,9.214640,9.920844,10.699744,7.944169,5.325128,0.0,0.0,0.0,0.0,...,0.0,0.0,93.75,91.150,95.225,96.40,97.725,93.425,100.000,96.68,96.750,99.62,96.650,97.450,0.0,0.0,0.0,Arica y Parinacota,Camarones,1233.0,0.0,0.0,0.0,0.0,0.0,87.50,61.975,96.900,95.42,95.625,99.225,98.450,94.50,96.150,95.04,95.950,96.875,0.0,0.0,0.0
2,Arica y Parinacota,General Lagos,15202.0,810.0,0.0,0.000000,0.000000,0.000,0.333333,35.222222,63.500,64.000000,64.000000,64.000,64.000000,64.250,65.000000,69.666667,78.444444,79.500,86.888889,87.000000,88.000,88.000000,Arica y Parinacota,General Lagos,810.0,1.352493,0.990023,0.953350,0.926154,0.796030,0.758809,1.180513,0.720099,0.811795,0.0,0.0,0.0,0.0,...,0.0,0.0,100.00,100.000,91.675,100.00,75.000,75.000,75.000,53.34,67.150,97.50,90.825,94.225,0.0,0.0,0.0,Arica y Parinacota,General Lagos,810.0,0.0,0.0,0.0,0.0,0.0,25.00,87.775,66.675,80.00,41.650,75.000,58.350,60.00,61.050,97.14,96.075,97.050,0.0,0.0,0.0
3,Arica y Parinacota,Putre,15201.0,2515.0,0.0,0.000000,0.000000,4.500,27.333333,50.444444,60.500,70.222222,72.444444,73.000,81.000000,117.000,128.333333,156.888889,169.555556,182.625,195.000000,200.777778,207.625,214.000000,Arica y Parinacota,Putre,2515.0,3.936189,2.406134,2.915727,3.678952,3.983215,3.855582,4.908582,5.249672,5.750949,0.0,0.0,0.0,0.0,...,0.0,0.0,99.00,99.250,98.300,97.26,98.825,87.675,99.025,99.36,98.750,93.70,90.325,89.725,0.0,0.0,0.0,Arica y Parinacota,Putre,2515.0,0.0,0.0,0.0,0.0,0.0,89.45,94.000,93.900,97.48,98.275,95.150,82.875,94.46,87.275,95.94,90.325,87.525,0.0,0.0,0.0
4,Tarapacá,Alto Hospicio,1107.0,129999.0,0.0,17.000000,338.666667,1433.500,2250.000000,2946.000000,3571.875,4006.666667,4394.888889,4856.500,6370.777778,8229.625,9518.888889,11301.00000

Creamos un df que será el que contiene toda la información y eliminamos las columnas duplicadas de región.

In [ ]:
df_master = df_merged
df_master.drop(columns=['Region_y'], inplace=True)
df_master.drop(columns=['Comuna_y'], inplace=True)
dedup_names = pd.io.parsers.ParserBase({'names':df_master.columns})._maybe_dedup_names(df_master.columns)
df_master.set_axis(dedup_names, axis=1, inplace=True)
df_master.drop(columns=['Region_x.1', 'Region_x.2', 'Comuna_x.1', 'Comuna_x.2'], inplace=True)
df_master.rename(columns={'Region_x': 'Region', 'Comuna_x': 'Comuna'}, inplace=True)

In [ ]:
# Vemos que ya no hay comunas que empiecen con "Desconocido"
df_merged[df_merged['Comuna'].str.startswith('Desconocido')]

,Region,Comuna,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,mobility_2020-12,mobility_2021-01,mobility_2021-02,mobility_2021-03,mobility_2021-04,mobility_2021-05,...,test_coverage_2020-04,test_coverage_2020-05,test_coverage_2020-06,test_coverage_2020-07,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,test_coverage_2021-08,test_coverage_2021-09,test_coverage_2021-10,Poblacion_y.2,BAC_2020-03,BAC_2020-04,BAC_2020-05,BAC_2020-06,BAC_2020-07,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07,BAC_2021-08,BAC_2021-09,BAC_2021-10


In [ ]:
# No hay nulos
rows_with_NaN = df_merged[df_merged.isnull().any(axis=1)]
rows_with_NaN

,Region,Comuna,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,mobility_2020-12,mobility_2021-01,mobility_2021-02,mobility_2021-03,mobility_2021-04,mobility_2021-05,...,test_coverage_2020-04,test_coverage_2020-05,test_coverage_2020-06,test_coverage_2020-07,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,test_coverage_2021-08,test_coverage_2021-09,test_coverage_2021-10,Poblacion_y.2,BAC_2020-03,BAC_2020-04,BAC_2020-05,BAC_2020-06,BAC_2020-07,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07,BAC_2021-08,BAC_2021-09,BAC_2021-10


In [ ]:
df_2 = df_master.replace(0,np.nan).dropna(axis=1,how="all")
df_3 = df_2.replace(np.nan,0).dropna(axis=1,how="all")
df_master = df_3
df_master.drop(columns=['Poblacion_y.1'])
df_master

,Region,Comuna,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
0,Arica y Parinacota,Arica,15101.0,247552.0,6.0,112.909091,406.888889,1155.875,3159.444444,6042.000000,7737.250,9294.222222,10332.222222,10842.125,11984.444444,14315.000,17066.000000,20570.111111,23859.000000,26726.375,28812.000000,29463.666667,29905.000,30671.444444,247552.0,5.394165,4.092249,4.283068,4.973705,5.049779,5.235622,5.868555,5.983744,6.312275,247552.0,13.4,36.111111,89.000000,133.375,166.777778,...,247552.0,11.55,7.625,7.200,3.48,2.550,5.650,7.050,6.34,6.950,5.68,5.800,3.575,1.7,247552.0,95.35,95.825,94.800,94.66,94.400,93.675,95.625,94.36,92.075,93.78,88.825,90.650,247552.0,60.60,66.975,64.700,73.38,78.525,81.650,83.025,84.74,85.300,87.16,86.425,87.700
1,Arica y Parinacota,Camarones,15102.0,1233.0,0.0,0.000000,0.000000,0.000,8.666667,24.333333,26.375,27.888889,28.000000,27.500,28.555556,30.375,39.666667,50.777778,57.222222,61.375,65.444444,67.444444,68.500,70.888889,1233.0,19.541699,7.755541,6.572208,8.568462,9.214640,9.920844,10.699744,7.944169,5.325128,1233.0,0.0,0.000000,0.888889,1.000,1.000000,...,1233.0,2.40,12.500,0.375,0.00,0.000,1.425,1.675,2.50,3.825,1.92,2.675,1.550,0.0,1233.0,93.75,91.150,95.225,96.40,97.725,93.425,100.000,96.68,96.750,99.62,96.650,97.450,1233.0,87.50,61.975,96.900,95.42,95.625,99.225,98.450,94.50,96.150,95.04,95.950,96.875
2,Arica y Parinacota,General Lagos,15202.0,810.0,0.0,0.000000,0.000000,0.000,0.333333,35.222222,63.500,64.000000,64.000000,64.000,64.000000,64.250,65.000000,69.666667,78.444444,79.500,86.888889,87.000000,88.000,88.000000,810.0,1.352493,0.990023,0.953350,0.926154,0.796030,0.758809,1.180513,0.720099,0.811795,810.0,0.0,0.000000,0.222222,1.000,1.000000,...,810.0,25.00,5.275,0.000,0.00,0.000,0.000,8.325,0.00,18.275,12.50,3.675,8.825,0.0,810.0,100.00,100.000,91.675,100.00,75.000,75.000,75.000,53.34,67.150,97.50,90.825,94.225,810.0,25.00,87.775,66.675,80.00,41.650,75.000,58.350,60.00,61.050,97.14,96.075,97.050
3,Arica y Parinacota,Putre,15201.0,2515.0,0.0,0.000000,0.000000,4.500,27.333333,50.444444,60.500,70.222222,72.444444,73.000,81.000000,117.000,128.333333,156.888889,169.555556,182.625,195.000000,200.777778,207.625,214.000000,2515.0,3.936189,2.406134,2.915727,3.678952,3.983215,3.855582,4.908582,5.249672,5.750949,2515.0,0.0,0.000000,0.000000,0.000,0.000000,...,2515.0,6.20,4.825,1.275,0.24,0.000,3.350,7.300,2.34,7.675,1.74,2.400,0.975,20.0,2515.0,99.00,99.250,98.300,97.26,98.825,87.675,99.025,99.36,98.750,93.70,90.325,89.725,2515.0,89.45,94.000,93.900,97.48,98.275,95.150,82.875,94.46,87.275,95.94,90.325,87.525
4,Tarapacá,Alto Hospicio,1107.0

Hacemos una descripción de los datos
 a usar

In [ ]:
df_master.describe()

,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,deceased_2020-11,deceased_2020-12,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
count,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,...,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.00000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.00000,344.000000,344.00000,344.000000,344.000000
mean,9019.200581,56562.534884,5.630814,22.123943,130.935724,536.087209,991.599483,1206.832687,1391.996366,1561.179587,1714.636951,1874.363735,2155.018411,2528.626817,2996.215116,3736.562339,4423.124677,5113.384448,5496.968023,5632.138243,5697.401163,5783.443475,56562.534884,6.216846,4.986945,5.041564,5.079379,5.601002,6.174270,6.968629,7.377583,7.965244,56562.534884,12.139535,24.450258,31.379199,36.031613,40.271964,44.909561,48.387718,...,56562.534884,7.006977,6.262282,6.111846,4.788488,6.381613,9.588953,10.003198,11.417791,13.243823,12.435174,11.100581,5.656323,1.393605,56562.534884,89.125581,91.92064,93.405233,93.733488,93.622238,91.985610,94.560465,94.411628,93.462573,91.032267,88.718387,87.587500,56562.534884,54.929070,56.267151,58.826744,59.988721,61.715698,60.549419,64.497529,63.50750,64.697674,67.26436,66.104506,68.369985
std,3823.451166,89102.350327,18.997477,56.790865,339.454712,1399.860820,2377.910427,2684.032176,2912.474765,3105.640476,3271.538333,3465.479206,3829.183794,4333.177828,4984.805655,6117.032714,7188.307547,8242.588237,8798.899973,8997.492915,9103.872267,9253.390700,89102.350327,3.448930,2.798961,2.704601,2.685504,3.024791,3.465312,4.144899,4.556876,4.925763,89102.350327,32.731078,62.661557,76.203552,83.763135,90.503960,97.407013,102.028276,...,89102.350327,5.688672,4.349964,6.611647,3.955982,5.076887,5.594283,4.967489,5.662282,6.078371,5.985183,5.130140,3.410303,1.952681,89102.350327,14.403157,9.32606,7.548798,7.097548,7.196332,6.907376,7.462136,9.633144,8.445428,9.819404,11.590342,12.066665,89102.350327,19.765269,16.153351,16.416445,15.305237,15.899684,16.283119,15.080041,14.41485,14.

Dividimos las columas correspondientes por población

In [ ]:
for i, row in enumerate(df_master.iterrows()):
  for p, column in enumerate(df_master):
    if 'cases' in column or 'deceased' in column:
      cant_poblacion = df_master.loc[[i],['Poblacion_x']]
      df_master.at[i, column]= df_master.iloc[[i],[p]].squeeze() / cant_poblacion.squeeze()  
  

In [ ]:
df_master.describe()

,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,deceased_2020-11,deceased_2020-12,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
count,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,...,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.00000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.00000,344.000000,344.00000,344.000000,344.000000
mean,9019.200581,56562.534884,0.000044,0.000262,0.001262,0.004946,0.010249,0.013716,0.016946,0.020497,0.023716,0.026974,0.032751,0.040773,0.049799,0.062727,0.075091,0.087499,0.094899,0.097482,0.098596,0.099907,56562.534884,6.216846,4.986945,5.041564,5.079379,5.601002,6.174270,6.968629,7.377583,7.965244,56562.534884,0.000104,0.000221,0.000309,0.000377,0.000438,0.000507,0.000562,...,56562.534884,7.006977,6.262282,6.111846,4.788488,6.381613,9.588953,10.003198,11.417791,13.243823,12.435174,11.100581,5.656323,1.393605,56562.534884,89.125581,91.92064,93.405233,93.733488,93.622238,91.985610,94.560465,94.411628,93.462573,91.032267,88.718387,87.587500,56562.534884,54.929070,56.267151,58.826744,59.988721,61.715698,60.549419,64.497529,63.50750,64.697674,67.26436,66.104506,68.369985
std,3823.451166,89102.350327,0.000168,0.000509,0.001857,0.006436,0.011243,0.013129,0.014052,0.014426,0.015005,0.016038,0.018009,0.020697,0.022007,0.023481,0.025782,0.027579,0.028581,0.029037,0.029158,0.029103,89102.350327,3.448930,2.798961,2.704601,2.685504,3.024791,3.465312,4.144899,4.556876,4.925763,89102.350327,0.000201,0.000348,0.000419,0.000456,0.000477,0.000498,0.000509,...,89102.350327,5.688672,4.349964,6.611647,3.955982,5.076887,5.594283,4.967489,5.662282,6.078371,5.985183,5.130140,3.410303,1.952681,89102.350327,14.403157,9.32606,7.548798,7.097548,7.196332,6.907376,7.462136,9.633144,8.445428,9.819404,11.590342,12.066665,89102.350327,19.765269,16.153351,16.416445,15.305237,15.899684,16.283119,15.080041,14.41485,14.573193,14.15872,14.193924,14.482872
min,1101.000000,211.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

El dataframe final cuenta con 90 columnas y 344 filas.

Para ver un poco la distribución de los datos, vamos a mostrar la evolución de los casos totales.

Obtenemos el dataframe para hacer el análisis de la región metropolitana. 

In [ ]:
df_master_metropolitana = df_master[df_master['Region'] == 'Metropolitana']
df_master_metropolitana

,Region,Comuna,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
82,Metropolitana,Alhue,13502.0,7405.0,0.000000,0.000000,0.000150,0.004254,0.019281,0.033596,0.045780,0.050416,0.054333,0.055689,0.056313,0.057984,0.063306,0.079286,0.088214,0.097367,0.104524,0.107060,0.107529,0.108890,7405.0,0.109454,0.063551,0.058146,0.052381,0.062935,0.081910,0.093184,0.136668,0.083333,7405.0,0.000081,0.000345,0.000510,0.000625,0.000750,...,7405.0,10.30,11.000,4.625,2.50,0.850,3.375,7.325,11.58,11.875,5.96,7.200,4.775,4.0,7405.0,81.55,85.850,92.025,95.70,87.250,86.000,93.775,94.78,89.100,59.26,62.000,60.700,7405.0,80.05,75.525,71.725,69.50,76.200,67.775,67.625,63.08,74.475,68.34,76.725,79.425
83,Metropolitana,Buin,13402.0,109641.0,0.000064,0.000307,0.002203,0.009980,0.019080,0.022795,0.025800,0.028353,0.030001,0.031600,0.034385,0.038397,0.044763,0.056371,0.068690,0.085428,0.094621,0.097603,0.099428,0.101029,109641.0,6.959855,6.230245,6.065094,5.666719,6.023731,6.857551,7.575264,8.360444,9.627637,109641.0,0.000296,0.000592,0.000736,0.000832,0.000954,...,109641.0,7.35,6.075,3.975,3.34,4.625,6.750,9.500,11.76,13.450,11.52,12.050,4.775,2.1,109641.0,86.10,91.875,90.975,90.30,92.975,87.100,92.475,90.22,89.600,85.62,81.750,86.925,109641.0,49.90,56.475,58.125,50.68,55.925,57.050,59.450,55.86,56.975,65.22,61.600,61.675
84,Metropolitana,Calera de Tango,13403.0,28525.0,0.000210,0.000357,0.001317,0.007778,0.018027,0.020769,0.022805,0.023722,0.024739,0.026144,0.027882,0.030324,0.035653,0.049126,0.062421,0.073659,0.081726,0.084460,0.086157,0.088628,28525.0,8.838029,9.207986,6.843134,7.025447,8.467747,9.127492,10.039020,11.698400,11.735486,28525.0,0.000042,0.000210,0.000304,0.000386,0.000448,...,28525.0,6.15,3.425,2.350,2.32,2.700,4.000,3.400,8.94,14.900,12.18,11.525,7.650,2.1,28525.0,92.65,91.625,89.875,90.72,90.800,89.100,89.975,94.44,94.900,88.30,91.100,92.350,28525.0,64.40,42.425,49.425,41.62,49.325,53.525,54.750,51.28,57.150,58.72,55.200,52.175
85,Metropolitana,Cerrillos,13102.0,88956.0,0.000000,0.000368,0.005242,0.018165,0.031235,0.035029,0.037145,0.038851,0.040019,0.041515,0.043821,0.046745,0.052874,0.062317,0.072513,0.085680,0.092345,0.094291,0.095744,0.097475,88956.0,10.947413,8.566077,8.891126,7.689902,11.039027,10.875165,12.618182,12.836347,13.125003,88956.0,0.000339,0.000719,0.000909,0.001067,0.001194,...,88956.0,3.95,4.175,3.575,1.94,3.850,6.000,6.225,9.46,9.550,11.92,11.925,5.725,2.1,88956.0,91.85,93.400,95.000,94.24,93.525,93.850,95.250,86.92,75.150,70.90,69.800,71.300,88956.0,66.95,64.125,63.800,67.18,69.825,65.475,73.725,68.34,73.875,73.30,71.450,71.325
86,Metropolitana,Cerro 

In [ ]:
df_master_metropolitana.describe()

,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,deceased_2020-11,deceased_2020-12,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,...,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,13238.076923,156251.384615,0.000125,0.000425,0.003798,0.016969,0.030101,0.034150,0.036716,0.038459,0.040040,0.041696,0.044314,0.047590,0.053122,0.064885,0.077190,0.090995,0.097956,0.100448,0.101930,0.103897,156251.384615,8.710154,7.191718,6.928389,6.936707,7.770475,8.648672,10.127783,11.097427,11.767577,156251.384615,0.000435,0.000846,0.001024,0.001123,0.001206,0.001294,0.001354,...,156251.384615,5.819231,4.875481,3.799038,3.183846,4.078365,6.420673,6.963462,10.306538,15.104808,15.323846,13.995673,6.427404,2.713462,156251.384615,89.820192,91.415865,90.664423,90.687308,90.254327,89.159615,92.384135,93.176154,89.991346,83.346923,80.986538,80.151442,156251.384615,56.924038,52.404327,53.074038,53.460385,55.522596,53.476923,61.153846,56.294231,57.430288,58.879615,57.268269,60.921154
std,178.051309,135806.806340,0.000164,0.000243,0.002019,0.006922,0.009813,0.010428,0.010949,0.011228,0.011388,0.011539,0.011771,0.012151,0.012510,0.014427,0.017259,0.019918,0.021056,0.021259,0.021308,0.021428,135806.806340,3.037806,1.901190,1.720868,1.744386,2.032300,2.385721,2.944095,3.242271,3.569993,135806.806340,0.000242,0.000390,0.000449,0.000479,0.000499,0.000529,0.000556,...,135806.806340,1.984453,1.789611,1.155878,1.062797,1.160133,1.497937,1.717695,2.173183,3.737499,4.705878,3.549150,1.890471,1.107555,135806.806340,4.680385,3.003328,3.164534,2.970794,3.592284,3.382970,2.218726,2.424215,5.089103,9.654211,10.312418,10.067280,135806.806340,13.422291,10.212656,8.779173,10.607430,10.587024,10.864891,9.542637,9.337091,10.199205,11.259521,11.968484,11.571947
min,13101.000000,7405.000000,0.000000,0.000000,0.000150,0.004254,0.014433,0.016143,0.017267,0.018366,0.019513,0.021203,0.024697,0.027624,0.031742,0

Este dataframe cuenta también con 90 columnas, pero solo 52 filas que corresponden a las comunas de la región metropolitana. 

Como para la siguiente sección queremos reducir dimensionalidad y luego clusterizar, queremos encontrar un valor optimo de clusters de KMeans. Para eso definimos la siguiente función que elige según el SilhouetteScore.

In [ ]:
from sklearn.metrics import silhouette_score

def best_silhouette_score_clusters(X, k=30):
    j = 0
    max_score = -1
    for i in range(2, k + 1):
        kmeans = KMeans(n_clusters=i)
        pca_clusters = kmeans.fit_predict(X)
        score = silhouette_score(X, pca_clusters)
        if score > max_score:
            max_score = score
            j = i

    return (j, max_score)

def top_silhouette_score_clusters(X, k=30):
    clusters = []
    for i in range(2, k + 1):
        kmeans = KMeans(n_clusters=i)
        pca_clusters = kmeans.fit_predict(X)
        score = silhouette_score(X, pca_clusters)
        clusters.append((i, score))

    return sorted(clusters, key=lambda x: -x[1])[:4]

## 4. Análisis a nivel país

Definimos el X que usaremos de aquí en adelante. Nos quedamos solo con los valores numéricos del df

In [ ]:
X, y = df_master.drop(columns=['Comuna', 'Region']), df_master[['Region']]
X

,Codigo comuna,Poblacion_x,total_cases_2020-03,total_cases_2020-04,total_cases_2020-05,total_cases_2020-06,total_cases_2020-07,total_cases_2020-08,total_cases_2020-09,total_cases_2020-10,total_cases_2020-11,total_cases_2020-12,total_cases_2021-01,total_cases_2021-02,total_cases_2021-03,total_cases_2021-04,total_cases_2021-05,total_cases_2021-06,total_cases_2021-07,total_cases_2021-08,total_cases_2021-09,total_cases_2021-10,Poblacion_y,mobility_2020-03,mobility_2020-04,mobility_2020-05,mobility_2020-06,mobility_2020-07,mobility_2020-08,mobility_2020-09,mobility_2020-10,mobility_2020-11,Poblacion_x.1,deceased_2020-06,deceased_2020-07,deceased_2020-08,deceased_2020-09,deceased_2020-10,deceased_2020-11,deceased_2020-12,...,Poblacion_y.1,PRC_positivity_2020-08,PRC_positivity_2020-09,PRC_positivity_2020-10,PRC_positivity_2020-11,PRC_positivity_2020-12,PRC_positivity_2021-01,PRC_positivity_2021-02,PRC_positivity_2021-03,PRC_positivity_2021-04,PRC_positivity_2021-05,PRC_positivity_2021-06,PRC_positivity_2021-07,PRC_positivity_2021-10,Poblacion_x.2,test_coverage_2020-08,test_coverage_2020-09,test_coverage_2020-10,test_coverage_2020-11,test_coverage_2020-12,test_coverage_2021-01,test_coverage_2021-02,test_coverage_2021-03,test_coverage_2021-04,test_coverage_2021-05,test_coverage_2021-06,test_coverage_2021-07,Poblacion_y.2,BAC_2020-08,BAC_2020-09,BAC_2020-10,BAC_2020-11,BAC_2020-12,BAC_2021-01,BAC_2021-02,BAC_2021-03,BAC_2021-04,BAC_2021-05,BAC_2021-06,BAC_2021-07
0,15101.0,247552.0,0.000024,0.000456,0.001644,0.004669,0.012763,0.024407,0.031255,0.037545,0.041738,0.043797,0.048412,0.057826,0.068939,0.083094,0.096380,0.107963,0.116388,0.119020,0.120803,0.123899,247552.0,5.394165,4.092249,4.283068,4.973705,5.049779,5.235622,5.868555,5.983744,6.312275,247552.0,0.000054,0.000146,0.000360,0.000539,0.000674,0.000816,0.000896,...,247552.0,11.55,7.625,7.200,3.48,2.550,5.650,7.050,6.34,6.950,5.68,5.800,3.575,1.7,247552.0,95.35,95.825,94.800,94.66,94.400,93.675,95.625,94.36,92.075,93.78,88.825,90.650,247552.0,60.60,66.975,64.700,73.38,78.525,81.650,83.025,84.74,85.300,87.16,86.425,87.700
1,15102.0,1233.0,0.000000,0.000000,0.000000,0.000000,0.007029,0.019735,0.021391,0.022619,0.022709,0.022303,0.023159,0.024635,0.032171,0.041182,0.046409,0.049777,0.053077,0.054699,0.055556,0.057493,1233.0,19.541699,7.755541,6.572208,8.568462,9.214640,9.920844,10.699744,7.944169,5.325128,1233.0,0.000000,0.000000,0.000721,0.000811,0.000811,0.000811,0.001115,...,1233.0,2.40,12.500,0.375,0.00,0.000,1.425,1.675,2.50,3.825,1.92,2.675,1.550,0.0,1233.0,93.75,91.150,95.225,96.40,97.725,93.425,100.000,96.68,96.750,99.62,96.650,97.450,1233.0,87.50,61.975,96.900,95.42,95.625,99.225,98.450,94.50,96.150,95.04,95.950,96.875
2,15202.0,810.0,0.000000,0.000000,0.000000,0.000000,0.000412,0.043484,0.078395,0.079012,0.079012,0.079012,0.079012,0.079321,0.080247,0.086008,0.096845,0.098148,0.107270,0.107407,0.108642,0.108642,810.0,1.352493,0.990023,0.953350,0.926154,0.796030,0.758809,1.180513,0.720099,0.811795,810.0,0.000000,0.000000,0.000274,0.001235,0.001235,0.001235,0.001235,...,810.0,25.00,5.275,0.000,0.00,0.000,0.000,8.325,0.00,18.275,12.50,3.675,8.825,0.0,810.0,100.00,100.000,91.675,100.00,75.000,75.000,75.000,53.34,67.150,97.50,90.825,94.225,810.0,25.00,87.775,66.675,80.00,41.650,75.000,58.350,60.00,61.050,97.14,96.075,97.050
3,15201.0,2515.0,0.000000,0.000000,0.000000,0.001789,0.010868,0.020057,0.024056,0.027921,0.028805,0.029026,0.032207,0.046521,0.051027,0.062381,0.067418,0.072614,0.077535,0.079832,0.082555,0.085089,2515.0,3.936189,2.406134,2.915727,3.678952,3.983215,3.855582,4.908582,5.249672,5.750949,2515.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2515.0,6.20,4.825,1.275,0.24,0.000,3.350,7.300,2.34,7.675,1.74,2.400,0.975,20.0,2515.0,99.00,99.250,98.300,97.26,98.825,87.675,99.025,99.36,98.750,93.70,90.325,89.725,2515.0,89.45,94.000,93.900,97.48,98.275,95.150,82.875,94.46,87.275,95.94,90.325,87.525
4,1107.0,129999.0,0.000000,0.000131,0.002605,0.011027,0.

### 4.1 Usando PCA
Primero vamos a hacer reducción de dimensionalidad a dos dimensiones con PCA para graficar

In [ ]:
# Estandarizamos
std_scaler = StandardScaler()
X_std = std_scaler.fit_transform(X)

# Queremos reducir a las dimensiones a 2 
pca_2 = PCA(n_components=2)

# Guardamos el dataset pero ahora en su versión reducida en 2 dimensiones
X_PCA = pca_2.fit_transform(X_std)
pca_2.explained_variance_ratio_

array([0.26347949, 0.14232497])

Probamos clusterizando con **kmeans**

In [ ]:
best_silhouette_score_clusters(X_PCA)

(2, 0.5092307149511924)

A continuación un listado de los 4 mejores silhouette score para dos dimensiones

In [ ]:
top_silhouette_score_clusters(X_PCA)

[(2, 0.5092307149511924),
 (4, 0.3937331960139819),
 (29, 0.3768170872758968),
 (3, 0.37622815764458145)]

In [ ]:
# el mejor score es con 2 clusters para kmeans

kmeans2 = KMeans(n_clusters=2)
pca_clusters2 = kmeans2.fit_predict(X_PCA)
pca_clusters2

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Graficamos

In [ ]:
df_pca_kmeans = pd.concat([pd.DataFrame(X_PCA), pd.DataFrame(pca_clusters2), pd.DataFrame(y), df_master['Comuna']], axis=1)
df_pca_kmeans.set_axis(['dim1', 'dim2', 'cluster', 'region', 'comuna'], axis=1, inplace=True)
df_pca_kmeans

,dim1,dim2,cluster,region,comuna
0,3.791244,1.045015,0,Arica y Parinacota,Arica
1,-2.488876,7.377346,1,Arica y Parinacota,Camarones
2,3.461578,-1.568894,1,Arica y Parinacota,General Lagos
3,-5.888086,3.277928,1,Arica y Parinacota,Putre
4,1.597426,-0.009610,1,Tarapacá,Alto Hospicio
...,...,...,...,...,...
339,10.273062,-6.972609,0,Magallanes,Punta Arenas
340,-5.685371,-1.163658,1,Magallanes,Rio Verde
341,-0.374341,-8.813634,1,Magallanes,San Gregorio
342,-2.982572,-6.918509,1,Magallanes,Timaukel


In [ ]:
px.scatter(df_pca_kmeans, x='dim1', y='dim2', hover_name='comuna', hover_data=['region'], color='cluster')

A continuación utilizamos PCA manteniendo el 95% de varianza

In [ ]:
# Estandarizamos
std_scaler = StandardScaler()
X_std = std_scaler.fit_transform(X)

# Queremos reducir manteniendo el 95%
pca = PCA(n_components=0.95)

# Guardamos el dataset pero ahora en su versión reducida
X_95 = pca.fit_transform(X_std)

print(X_95.shape)
pca.explained_variance_ratio_


(344, 26)


array([0.26347949, 0.14232497, 0.10887649, 0.08733959, 0.06175616,
       0.05044536, 0.02906438, 0.0242371 , 0.02267624, 0.02054941,
       0.01843255, 0.0151571 , 0.01341475, 0.01125614, 0.01072001,
       0.00907667, 0.00821724, 0.00810274, 0.00773285, 0.00674831,
       0.00637339, 0.00564579, 0.00533025, 0.00483152, 0.00456709,
       0.00420306])

Nos quedamos con 26 dimensiones para obtener el 0.95 de varianza 

In [ ]:
best_silhouette_score_clusters(X_95)

(2, 0.30759808190623933)

In [ ]:
top_silhouette_score_clusters(X_95)

[(2, 0.30434703046986034),
 (3, 0.18634479601206275),
 (7, 0.15691386930129497),
 (6, 0.15585546022116004)]

In [ ]:
# nos quedamos con 2 clusters que es el mejor score
kmeans95_2 = KMeans(n_clusters=2)
pca_clusters95 = kmeans95_2.fit_predict(X_95)
pca_clusters95

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
df_pca_kmeans = pd.concat([pd.DataFrame(X_95), pd.DataFrame(pca_clusters95), pd.DataFrame(y), df_master['Comuna']], axis=1)
df_pca_kmeans.set_axis(["dim" + str(i + 1) for i in range(X_95.shape[1])] + ["cluster", "region", "comuna"], axis=1, inplace=True)
df_pca_kmeans

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,dim11,dim12,dim13,dim14,dim15,dim16,dim17,dim18,dim19,dim20,dim21,dim22,dim23,dim24,dim25,dim26,cluster,region,comuna
0,3.791244,1.045015,4.827509,0.153688,2.415301,-4.086410,-1.462939,1.053183,-0.621954,-0.724493,-1.379022,-0.675999,0.511723,-1.072265,0.485404,-0.496382,-0.716943,-0.336419,0.809532,-0.664559,0.350160,0.334162,0.031284,-0.207332,-0.017903,-0.630117,1,Arica y Parinacota,Arica
1,-2.488876,7.377346,4.603310,-3.153702,-0.381580,1.336256,-0.657055,1.345265,0.753632,-0.338354,-0.658674,-1.406805,1.004445,0.418419,1.363651,-1.738852,-0.276854,0.726979,0.272120,-1.113382,0.617174,0.809547,-1.053395,-0.075286,0.300894,-0.346065,0,Arica y Parinacota,Camarones
2,3.461578,-1.568894,6.747169,4.880477,-1.642631,2.734141,-0.662934,5.478762,-3.112027,0.349144,-1.412711,-0.597535,-3.169556,-1.790869,0.504117,-1.268729,1.919848,-2.213039,3.073732,-1.132644,4.165269,-1.907237,-2.235339,-1.833865,0.706854,-1.525068,1,Arica y Parinacota,General Lagos
3,-5.888086,3.277928,6.714476,-0.867313,0.418745,-0.630275,3.394784,0.512527,-4.594230,2.624517,0.586544,-0.031673,1.461945,-0.195299,5.319621,5.648732,-2.964351,-0.478549,-0.569389,-0.237919,0.413622,-0.510394,-0.877907,-1.295501,0.791764,-0.450021,0,Arica y Parinacota,Putre
4,1.597426,-0.009610,4.753089,-0.703149,1.935615,-1.523864,-0.541209,0.481721,-0.821568,-0.066375,1.397743,1.411990,-0.173672,0.157417,-0.639341,0.908479,1.462530,-0.121350,0.156273,0.809967,-0.223023,-0.182820,0.733881,0.087594,0.251120,0.107219,0,Tarapacá,Alto Hospicio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,10.273062,-6.972609,5.888839,-0.553500,5.823288,-0.090982,0.152468,6.115914,2.665581,-1.222764,-1.082659,-2.476628,-0.467674,-1.025520,1.055387,-0.565864,-1.231420,1.049412,-1.477531,0.311472,-0.155929,0.438807,0.212943,-0.238604,0.207750,-1.640068,1,Magallanes,Punta Arenas
340,-5.685371,-1.163658,-3.759476,14.439950,2.853279,7.033972,-2.810457,-3.519434,1.169673,1.659513,4.127862,2.969640,-1.802705,-3.568389,-0.571717,0.189724,-1.489954,-1.362535,1.778057,-2.810278,-0.569346,4.932097,-3.109756,-0.335214,0.015524,-0.161431,0,Magallanes,Rio Verde
341,-0.374341,-8.813634,-1.724352,0.601071,2.902164,2.482078,-2.889376,-4.213532,-0.735019,0.747446,8.393277,-4.829788,-1.198157,3.143081,2.770892,-1.291923,-0.748544,-2.278950,1.649253,-0.371051,2.299651,0.586745,3.197067,2.858697,2.172585,2.276877,0,Magallanes,San Gregorio
342,-2.982572,-6.918509,7.132091,5.821234,4.780349,2.795865,0.534261,4.107066,-0.443919,0.984347,-1.683475,3.084406,3.500245,5.717506,0.681563,-1.535317,-1.817559,1.925017,-2.008647,-2.187552,0.540737,1.978299,-0.482669,-0.607747,-1.413251,2.713391,0,Magallanes,Timaukel


## 5. Análisis Región Metropolitana

In [ ]:
X_metrop, y_metrop = df_master_metropolitana.drop(columns=['Comuna', 'Region']), df_master_metropolitana[['Comuna']]

### 5.1 Usando PCA

In [ ]:
# Estandarizamos
std_scaler1 = StandardScaler()
X_metrop_std = std_scaler1.fit_transform(X_metrop)

# Queremos reducir las dimensiones a 2 para graficar algo
pca_2 = PCA(n_components=2)

# Guardamos el dataset pero ahora en su versión reducida en 2 dimensiones
X_metrop_PCA = pca_2.fit_transform(X_metrop_std)
pca_2.explained_variance_ratio_

array([0.38969806, 0.16337092])

Probamos clusterizando con **kmeans**

In [ ]:
best_silhouette_score_clusters(X_metrop_PCA)

(3, 0.4747927951565718)

In [ ]:
top_silhouette_score_clusters(X_metrop_PCA)

[(3, 0.4747927951565718),
 (16, 0.46074067717936246),
 (2, 0.4602819145316686),
 (18, 0.4492940638390651)]

In [ ]:
# Elegimos 3 clusters porque es el que da mejor score

kmeans_3 = KMeans(n_clusters=3)
pca_clusters_metrop = kmeans_3.fit_predict(X_metrop_PCA)
pca_clusters_metrop

array([0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2,
       1, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 2, 0, 0, 2], dtype=int32)

Graficamos

In [ ]:
df_pca_kmeans_metrop = pd.concat([pd.DataFrame(X_metrop_PCA), pd.DataFrame(pca_clusters_metrop), y_metrop.reset_index()['Comuna']], axis=1)
df_pca_kmeans_metrop.set_axis(['dim1', 'dim2', 'cluster', 'comuna'], axis=1, inplace=True)

In [ ]:
px.scatter(df_pca_kmeans_metrop, x='dim1', y='dim2', hover_name='comuna', color='cluster')

Ahora calculamos PCA con 0.95 

In [ ]:
# Estandarizamos
std_scaler = StandardScaler()
X_std = std_scaler.fit_transform(X_metrop)

# Queremos reducir manteniendo el 95%
pca_95 = PCA(n_components=0.95)

# Guardamos el dataset pero ahora en su versión reducida
X_metrop_PCA95 = pca_95.fit_transform(X_metrop_std)


print(X_metrop_PCA95.shape)
pca_95.explained_variance_ratio_

(52, 15)


array([0.38969806, 0.16337092, 0.11085838, 0.06883086, 0.04798883,
       0.04437071, 0.03110676, 0.02020835, 0.01546496, 0.01472199,
       0.01081111, 0.01001903, 0.00860689, 0.00780103, 0.00625493])

Para obtener el 0.95 de varianza nos quedamos con 15 dimensiones

In [ ]:
best_silhouette_score_clusters(X_metrop_PCA95)

(3, 0.2513083807118407)

In [ ]:
top_silhouette_score_clusters(X_metrop_PCA95)

[(2, 0.2509995974297051),
 (3, 0.2434473813324963),
 (4, 0.20401941788951025),
 (6, 0.17804420706984858)]

In [ ]:
# usamos 4 clusters porque tiene un buen score y coincide con la implementación del plan paso a paso del gobierno
kmeans_4 = KMeans(n_clusters=4)
pca_clusters_metrop = kmeans_4.fit_predict(X_metrop_PCA95)
pca_clusters_metrop

array([0, 0, 0, 0, 1, 3, 1, 0, 1, 0, 0, 3, 1, 0, 1, 3, 1, 1, 2, 3, 2, 2,
       1, 1, 0, 3, 0, 0, 2, 0, 0, 1, 0, 1, 0, 2, 3, 3, 3, 1, 1, 1, 3, 1,
       1, 1, 0, 1, 3, 0, 3, 2], dtype=int32)

In [ ]:
df_pca_kmeans_metrop = pd.concat([pd.DataFrame(X_metrop_PCA95), pd.DataFrame(pca_clusters_metrop), y_metrop.reset_index()['Comuna']], axis=1)
df_pca_kmeans_metrop.set_axis(["dim" + str(i + 1) for i in range(X_metrop_PCA95.shape[1])] + ["cluster", "comuna"], axis=1, inplace=True)

## 6. Visualización en Mapas

### 6.1 Región Metropolitana

In [ ]:
# Agregamos la columna de codigo de comuna al resultado de aplicar PCA y Kmeans a los datos
df_pca_kmeans_metrop['CodigoComuna'] = df_master_metropolitana['Codigo comuna'].reset_index(drop=True).astype(int)
# Y pasamos los clusters obtenidos a strings para que plotly los considere como categorias discretas
df_pca_kmeans_metrop['cluster'] = df_pca_kmeans_metrop['cluster'].astype(str)

In [ ]:
rm_geojson = 'https://raw.githubusercontent.com/caracena/chile-geojson/master/13.geojson'
fig = px.choropleth_mapbox(df_pca_kmeans_metrop, geojson=rm_geojson, locations='CodigoComuna', color='cluster',
                           hover_data=["comuna"],
                           mapbox_style="carto-positron",
                           featureidkey="properties.cod_comuna",
                           zoom=7, center = {"lat": -33.4489, "lon": -70.6693})

fig.show()

### 6.2 Todas las Comunas de Chile

In [ ]:
# Agregamos la columna de codigo de comuna al resultado de aplicar PCA y Kmeans a los datos
df_pca_kmeans['CodigoComuna'] = df_master['Codigo comuna'].reset_index(drop=True).astype(int)
# Y pasamos los clusters obtenidos a strings para que plotly los considere como categorias discretas
df_pca_kmeans['cluster'] = df_pca_kmeans['cluster'].astype(str)
df_pca_kmeans

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,dim11,dim12,dim13,dim14,dim15,dim16,dim17,dim18,dim19,dim20,dim21,dim22,dim23,dim24,dim25,dim26,cluster,region,comuna,CodigoComuna
0,3.791244,1.045015,4.827509,0.153688,2.415301,-4.086410,-1.462939,1.053183,-0.621954,-0.724493,-1.379022,-0.675999,0.511723,-1.072265,0.485404,-0.496382,-0.716943,-0.336419,0.809532,-0.664559,0.350160,0.334162,0.031284,-0.207332,-0.017903,-0.630117,1,Arica y Parinacota,Arica,15101
1,-2.488876,7.377346,4.603310,-3.153702,-0.381580,1.336256,-0.657055,1.345265,0.753632,-0.338354,-0.658674,-1.406805,1.004445,0.418419,1.363651,-1.738852,-0.276854,0.726979,0.272120,-1.113382,0.617174,0.809547,-1.053395,-0.075286,0.300894,-0.346065,0,Arica y Parinacota,Camarones,15102
2,3.461578,-1.568894,6.747169,4.880477,-1.642631,2.734141,-0.662934,5.478762,-3.112027,0.349144,-1.412711,-0.597535,-3.169556,-1.790869,0.504117,-1.268729,1.919848,-2.213039,3.073732,-1.132644,4.165269,-1.907237,-2.235339,-1.833865,0.706854,-1.525068,1,Arica y Parinacota,General Lagos,15202
3,-5.888086,3.277928,6.714476,-0.867313,0.418745,-0.630275,3.394784,0.512527,-4.594230,2.624517,0.586544,-0.031673,1.461945,-0.195299,5.319621,5.648732,-2.964351,-0.478549,-0.569389,-0.237919,0.413622,-0.510394,-0.877907,-1.295501,0.791764,-0.450021,0,Arica y Parinacota,Putre,15201
4,1.597426,-0.009610,4.753089,-0.703149,1.935615,-1.523864,-0.541209,0.481721,-0.821568,-0.066375,1.397743,1.411990,-0.173672,0.157417,-0.639341,0.908479,1.462530,-0.121350,0.156273,0.809967,-0.223023,-0.182820,0.733881,0.087594,0.251120,0.107219,0,Tarapacá,Alto Hospicio,1107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,10.273062,-6.972609,5.888839,-0.553500,5.823288,-0.090982,0.152468,6.115914,2.665581,-1.222764,-1.082659,-2.476628,-0.467674,-1.025520,1.055387,-0.565864,-1.231420,1.049412,-1.477531,0.311472,-0.155929,0.438807,0.212943,-0.238604,0.207750,-1.640068,1,Magallanes,Punta Arenas,12101
340,-5.685371,-1.163658,-3.759476,14.439950,2.853279,7.033972,-2.810457,-3.519434,1.169673,1.659513,4.127862,2.969640,-1.802705,-3.568389,-0.571717,0.189724,-1.489954,-1.362535,1.778057,-2.810278,-0.569346,4.932097,-3.109756,-0.335214,0.015524,-0.161431,0,Magallanes,Rio Verde,12103
341,-0.374341,-8.813634,-1.724352,0.601071,2.902164,2.482078,-2.889376,-4.213532,-0.735019,0.747446,8.393277,-4.829788,-1.198157,3.143081,2.770892,-1.291923,-0.748544,-2.278950,1.649253,-0.371051,2.299651,0.586745,3.197067,2.858697,2.172585,2.276877,0,Magallanes,San Gregorio,12104
342,-2.982572,-6.918509,7.132091,5.821234,4.780349,2.795865,0.534261,4.107066,-0.443919,0.984347,-1.683475,3.084406,3.500245,5.717506,0.681563,-1.535317,-1.817559,1.925017,-2.008647,-2.187552,0.540737,1.978299,-0.482669,-0.607747,-1.413251,2.713391,0,Magallanes,Timaukel,12303


In [ ]:
chile_geojson = 'https://gist.githubusercontent.com/MarceloSaldias/8123ad8d4a78737123703159850ef8d0/raw/8cf696b51d1176aa1f5a4539eedfaf8a8c184259/All_Comunas.geojson'
fig = px.choropleth_mapbox(df_pca_kmeans, geojson=chile_geojson, locations='CodigoComuna', color='cluster',
                           hover_data=["comuna", "region"],
                           mapbox_style="carto-positron",
                           featureidkey="properties.cod_comuna",
                           zoom=3.5, center = {"lat": -40, "lon": -70.6693})

fig.update_layout(
    autosize=False,
    width=700,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=20,
        t=20,
        pad=1
    ),
)

fig.show()

## 7. Conclusiones

A partir de los resultados, podemos observar que el análisis pareciera tener coincidencia con lo que esperábamos. Cuando realizamos el análisis con todas las comunas vemos que las de regiones (no metropolitana) quedan graficadas muy cerca y en el mismo cluster o en clusters muy cercanos. Con respecto a las comunas de la región metropolitana, los clusters coinciden con respecto a su nivel socieconómico, cantidad de población y localidad, por ejemplo comunas como Providencia, La Reina, Lo Barnechea, Las Condes, Vitacura y Ñuñoa quedan en el mismo cluster de `Kmeans`, mientras que comunas rurales como Curacaví, Isla de Maipo, Paine, Talagante y Pirque, entre otras, están en otro cluster. 


### Que se puede hacer con este modelo?  

Primero, puede servir para estudiar implementaciones de medidas ya sean preventivas o reactivas en ciertas comunas representativas de los clusters, analizar como funcionan y en base a eso decidir si implementar en el resto de las comunas del cluster.

Además, si bien actualmente, la mayoría de las comunas se encuentra en fase 4, eventualmente con las nuevas variantes o con otras enfermedades como la influenza,esta agrupación de comunas podría servir para tomar mejores medidas preventivas. Cabe destacar que en este modelo está entrenado con datos de COVID-19, por lo que en caso de utilizarlo para otras enfermedades o variantes que puedan tener otro comportamiento, esto es una buena aproximación inicial ,pero con el tiempo se debería ir calibrando con los datos respectivos.

### Posibles aplicaciones

Teniendo los datos respectivos,  cualquier  caso de uso que tenga relación con comportamiento de comunas, por ejemplo votaciones, puntajes PTU, entre otros.


